Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [1]:
library('ISLR2')
library('tidymodels')


── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.5      ✔ recipes      1.0.10
✔ dials        1.2.1      ✔ rsample      1.2.1 
✔ dplyr        1.1.4      ✔ tibble       3.2.1 
✔ ggplot2      3.5.1      ✔ tidyr        1.3.1 
✔ infer        1.0.7      ✔ tune         1.2.1 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.1      ✔ workflowsets 1.1.0 
✔ purrr        1.0.2      ✔ yardstick    1.3.1 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Search for functions across packages at https://www.tidymodels.org/find/



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [2]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [3]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [4]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [5]:
mod = logistic_reg(mode = 'classification') |> set_engine('glm')

mod_fit = mod |> fit(default ~ income + balance, Default_train)

tidy(mod_fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.141166e+01,4.480818e-01,-25.467801,4.483851e-143
income,2.009934e-05,5.201940e-06,3.863816,1.116294e-04
balance,5.594747e-03,2.344502e-04,23.863268,7.374792e-126


Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [6]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9657778


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [7]:
folds = vfold_cv(Default_train, v =10, strata = 'default')

folds |> glimpse()

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [8]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97266667,10,0.0012417077,Preprocessor1_Model1
brier_class,binary,0.02204823,10,0.0008102308,Preprocessor1_Model1
roc_auc,binary,0.94685972,10,0.0049570688,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

The accuracy is slightly better, at 97.37 percent versus 96.68 percent with the naive model.


Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [9]:
rec = recipe(default ~ income + balance + student, data = Default_train) |>
    step_dummy(student) |>
    prep()

rec |> juice() |> head()

income,balance,default,student_Yes
<dbl>,<dbl>,<fct>,<dbl>
12106.135,817.1804,No,1
31767.139,1073.5492,No,0
35704.494,529.2506,No,0
38463.496,785.6559,No,0
7491.559,919.5885,No,1
24905.227,825.5133,No,0


In [10]:
mod = logistic_reg(mode = 'classification') |> set_engine('glm')

mod_stud_fit = mod |> fit(default ~ income + balance + student_Yes, juice(rec))

tidy(mod_stud_fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.092128e+01,5.161247e-01,-21.1601655,2.224405e-99
income,5.437597e-06,8.608849e-06,0.6316288,5.276294e-01
balance,5.734502e-03,2.437302e-04,23.5280723,2.105397e-122
student_Yes,-6.228835e-01,2.497165e-01,-2.4943629,1.261835e-02


In [11]:
# Note: needed to remove the recipe here as "fit_resamples" was not working with a trained recipe

mod |>
    fit_resamples(default ~ income + balance + student, folds) |>
    collect_metrics()


.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97322222,10,0.002104603,Preprocessor1_Model1
brier_class,binary,0.02166173,10,0.001457424,Preprocessor1_Model1
roc_auc,binary,0.94992152,10,0.006097116,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

The model accuracy (when including student) is very similar, but slightly lower at 97.32 percent versus 97.37 percent. 


Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [13]:
mod = logistic_reg(mode = 'classification') |> set_engine('glm')

mod_test_fit = mod |> fit(default ~ income + balance, Default_test)

tidy(mod_test_fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.002762e+01,1.2525064506,-8.0060412,1.184601e-15
income,8.595710e-06,0.0000172629,0.4979297,6.185336e-01
balance,4.930742e-03,0.0006281272,7.8499097,4.163370e-15


In [14]:
Default_test |>
    mutate(.pred = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.969


The accuracy on the test data is actually slightly lower than the naive model.